In [1]:
import pandas as pd


In [2]:
df=pd.read_csv('/home/miguel/IronHack/DataSets/GSAF5.csv', encoding='Windows-1252')

In [6]:
paises=(df[['Country','Location']].groupby('Country').count()).

paises

AttributeError: 'DataFrame' object has no attribute 'sort'